In [101]:
from google import genai
from google.genai import types

client = genai.Client()
messages = []

In [102]:
get_weather_function = {
        "name": "get_weather",
        "description": "해당 도시의 날씨를 조회하는 함수",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨 정보를 얻으려는 도시 이름"
                }
            },
            "required": ["city"]
        }
    }

# Configure the client and tools
tools = types.Tool(function_declarations=[get_weather_function])
config = types.GenerateContentConfig(tools=[tools])

In [ ]:
from google.genai import types

def get_weather(city):
#     """해당 도시의 날씨를 조회하는 함수"""
#     # 실제 구현은 여기에 추가 (예: API 호출)
    return f"{city}는 -3도 입니다."

# FunctionMap: 함수 이름과 실제 함수 객체를 매핑
FUNCTION_MAP = {
    "get_weather": get_weather,
    # 새로운 함수를 추가할 때마다 여기에 등록
}

def build_conversation():
    """대화 히스토리를 문자열로 변환"""
    conversation = ""
    # 시스템 프롬프트 추가
    conversation += "당신은 도움이 되는 AI 어시스턴트입니다. 사용자의 질문에 친절하고 정확하게 답변해주세요.\n"
    conversation += "- 일반적인 지식 질문에는 자신의 지식을 바탕으로 답변해주세요.\n"
    conversation += "- 날씨 정보가 필요한 경우에만 get_weather 함수를 사용하세요.\n"
    conversation += "- 함수를 사용할 필요가 없는 일반적인 질문에는 직접 답변해주세요.\n\n"
    
    for msg in messages:
        role = msg.get("role", "user")
        content = msg.get("content", "")
        function_call = msg.get("function_call")
        
        if role == "user":
            conversation += f"사용자: {content}\n\n"
        elif role == "assistant":
            if function_call:
                func_name = function_call.get("name", "")
                func_args = function_call.get("arguments", {})
                if content:
                    conversation += f"AI: {content}\n\n"
                conversation += f"AI가 {func_name} 함수를 호출하려고 합니다. 인자: {func_args}\n\n"
            else:
                conversation += f"AI: {content}\n\n"
        elif role == "function":
            function_name = msg.get("name", "")
            conversation += f"함수 {function_name} 실행 결과: {content}\n\n"
    
    # 마지막 메시지가 function role이면 함수 호출 지시 추가
    last_msg_role = messages[-1].get("role") if messages else None
    if last_msg_role == "function":
        conversation += "위 함수 실행 결과를 바탕으로 사용자에게 자연스러운 텍스트로 응답해주세요. 더 이상 함수를 호출하지 마세요.\n\n"
    
    return conversation

def call_gemini_api(conversation, use_tools):
    """Gemini API 호출"""
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=conversation,
            config=config if use_tools else None,
        )
        return response
    except Exception as e:
        print(f"API 호출 오류: {e}")
        return None

def parse_response(response):
    """응답에서 텍스트와 함수 호출 정보 추출"""
    if not response or not response.candidates:
        return [], None
    
    parts = response.candidates[0].content.parts
    text_parts = []
    function_call = None
    
    for part in parts:
        if hasattr(part, 'text') and part.text:
            text_parts.append(part.text)
        if hasattr(part, 'function_call') and part.function_call:
            function_call = part.function_call
    
    return text_parts, function_call

def execute_function(function_name, function_args):
    """함수 실행 및 결과를 messages에 추가"""
    if function_name not in FUNCTION_MAP:
        error_msg = f"알 수 없는 함수: {function_name}"
        print(f"Error: {error_msg}")
        messages.append({
            "role": "function",
            "name": function_name,
            "content": error_msg
        })
        return False
    
    try:
        function = FUNCTION_MAP[function_name]
        result = function(**function_args)
        print(f"Function result: {result}")
        
        messages.append({
            "role": "function",
            "name": function_name,
            "content": str(result)
        })
        return True
    except Exception as e:
        error_msg = f"함수 실행 중 오류 발생: {str(e)}"
        print(f"Error: {error_msg}")
        messages.append({
            "role": "function",
            "name": function_name,
            "content": error_msg
        })
        return False

def handle_function_call(function_call, text_parts):
    """함수 호출 처리"""
    function_name = function_call.name
    function_args = function_call.args
    
    print(f"Function to call: {function_name}")
    print(f"Arguments: {function_args}")
    
    # assistant의 함수 호출 요청을 messages에 저장
    assistant_content = {
        "role": "assistant",
        "content": "".join(text_parts) if text_parts else None,
        "function_call": {
            "name": function_name,
            "arguments": function_args
        }
    }
    messages.append(assistant_content)
    
    # 함수 실행
    execute_function(function_name, function_args)
    
    # 함수 결과를 받아서 다시 모델에 전달 (재귀 호출)
    call_ai()

def handle_text_response(text_parts):
    """텍스트 응답 처리"""
    response_text = "".join(text_parts)
    
    if response_text:
        messages.append({"role": "assistant", "content": response_text})
        print(f"AI: {response_text}")
    else:
        # 응답이 비어있는 경우 처리
        last_msg_role = messages[-1].get("role") if messages else None
        if last_msg_role == "function" and messages:
            last_function_result = messages[-1].get("content", "")
            if last_function_result:
                response_text = f"함수 실행 결과: {last_function_result}"
                messages.append({"role": "assistant", "content": response_text})
                print(f"AI: {response_text}")
            else:
                print("AI: (응답 없음)")
        else:
            print("AI: (응답 없음)")

def call_ai():
    """메인 함수: AI와의 대화 처리"""
    # 대화 히스토리 생성
    conversation = build_conversation()
    
    # 마지막 메시지가 function role이면 tools 없이 호출
    last_msg_role = messages[-1].get("role") if messages else None
    use_tools = (last_msg_role != "function")
    
    # API 호출
    response = call_gemini_api(conversation, use_tools)
    if not response:
        return
    
    # 응답 파싱
    text_parts, function_call = parse_response(response)
    
    # 함수 호출 또는 텍스트 응답 처리
    if function_call:
        handle_function_call(function_call, text_parts)
    else:
        handle_text_response(text_parts)

In [104]:
while True:
    ask = input("send a msg to the LLM: ")
    if ask == 'q' or ask == 'ㅂ':
        break   
    else:
        messages.append({"role": "user", "content": ask})
        print(f"User: {ask}")
        call_ai()
        print()  # 빈 줄 추가
